In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import tqdm as notebook_tqdm
from sklearn.metrics import r2_score,mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold


c:\Users\zxcas\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import random
import os
import numpy as np
import pandas as pd
from pycaret.regression import *
from pycaret.utils import version
import warnings
warnings.filterwarnings('ignore')
version()

home_path = r'C:\Users\zxcas\PythonWork\LGAimers'

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(SEED) # Seed 고정

data = pd.read_csv(home_path+'/data/train.csv').drop(["ID", "X_04", "X_23", "X_47", "X_48"], axis=1)
X = data.filter(regex="X")
y = data.filter(regex="Y")


train_01 = pd.concat([X, y['Y_01']], axis=1)
train_02 = pd.concat([X, y['Y_02']], axis=1)
train_03 = pd.concat([X, y['Y_03']], axis=1)
train_04 = pd.concat([X, y['Y_04']], axis=1)
train_05 = pd.concat([X, y['Y_05']], axis=1)
train_06 = pd.concat([X, y['Y_06']], axis=1)
train_07 = pd.concat([X, y['Y_07']], axis=1)
train_08 = pd.concat([X, y['Y_08']], axis=1)
train_09 = pd.concat([X, y['Y_09']], axis=1)
train_10 = pd.concat([X, y['Y_10']], axis=1)
train_11 = pd.concat([X, y['Y_11']], axis=1)
train_12 = pd.concat([X, y['Y_12']], axis=1)
train_13 = pd.concat([X, y['Y_13']], axis=1)
train_14 = pd.concat([X, y['Y_14']], axis=1)
test = pd.read_csv(home_path+"/data/test.csv").drop(["ID", "X_04", "X_23", "X_47", "X_48"], axis=1)
preds = {}

for idx in range(1, 15):
    train = locals()[f"train_{str(idx).zfill(2)}"]
    reg = setup(data = train, target = f"Y_{str(idx).zfill(2)}", silent=True)
    best4_model = compare_models(fold = 5, n_select = 5)
    best_tune = []
    for model in best4_model:
        best_tune.append(tune_model(model))
    blended = blend_models(estimator_list = best_tune, fold = 5)
    final_model = finalize_model(blended)
    prediction = predict_model(final_model, data = test)
    preds[f"Y_{str(idx).zfill(2)}"] = prediction["Label"]


In [2]:
# 평가 산식 : Normalized RMSE (NRMSE)
def lg_nrmse(y_true, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(y_true[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(y_true[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
# train_df = pd.read_csv('./new_data/pos_data.csv')
train_df = pd.read_csv('./data/train.csv')

: 

In [7]:
train_x =x= train_df.filter(regex='X') # Input : X Featrue
train_y =y= train_df.filter(regex='Y') # Output : Y Feature
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()

In [8]:
from sklearn.model_selection import train_test_split
train_x, x_test,train_y,y_test = train_test_split(train_x,train_y,test_size=0.1,random_state=42)
print(train_x.shape,train_y.shape)
print(x_test.shape,y_test.shape)

(35646, 56) (35646, 14)
(3961, 56) (3961, 14)


In [9]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer 

params = {"n_independent": [1, 2, 3], 
          "gamma" : [1.0,1.1,1.2],
          "n_shared": [1, 2, 3], 
          "lambda_sparse": [0.001, 0.0005]}

score = make_scorer(lg_nrmse, greater_is_better=False)

regressor = TabNetRegressor(verbose=4,seed=42)
# regressor.fit(X_train=train_x, y_train=train_y,
#             patience=10, max_epochs=100,
#             eval_set=[(x_test, y_test)],
#             eval_metric=['rmse'])

reg = GridSearchCV(regressor, params, cv=3,verbose=4, scoring=score)
rnd_search = reg.fit(train_x, train_y, eval_set = [(x_test, y_test)], eval_metric = ["rmse"], max_epochs=150, patience=20)
print(rnd_search.best_params_)
print(rnd_search.best_score_)

Device used : cuda
Device used : cuda
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Device used : cuda
epoch 0  | loss: 362.0128| val_0_rmse: 96.04707|  0:00:04s
epoch 4  | loss: 1.65222 | val_0_rmse: 67.38596|  0:00:07s
epoch 8  | loss: 1.55609 | val_0_rmse: 15.64253|  0:00:11s
epoch 12 | loss: 1.55143 | val_0_rmse: 8.09277 |  0:00:14s
epoch 16 | loss: 1.53738 | val_0_rmse: 1.58404 |  0:00:18s
epoch 20 | loss: 1.53637 | val_0_rmse: 1.28883 |  0:00:21s
epoch 24 | loss: 1.52349 | val_0_rmse: 1.21143 |  0:00:25s
epoch 28 | loss: 1.53553 | val_0_rmse: 1.20646 |  0:00:28s
epoch 32 | loss: 1.51972 | val_0_rmse: 1.20652 |  0:00:32s
epoch 36 | loss: 1.51938 | val_0_rmse: 1.20555 |  0:00:35s
epoch 40 | loss: 1.51429 | val_0_rmse: 1.20324 |  0:00:39s
epoch 44 | loss: 1.5063  | val_0_rmse: 1.20079 |  0:00:42s
epoch 48 | loss: 1.50713 | val_0_rmse: 1.20155 |  0:00:46s
epoch 52 | loss: 1.5042  | val_0_rmse: 1.20143 |  0:00:49s
epoch 56 | loss: 1.50197 | val_0_rmse: 1.20182 |  0:00:

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(train_x):
    X_train, X_valid = train_x[train_index], train_x[test_index]
    y_train, y_valid = train_y[train_index], train_y[test_index]
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=10, max_epochs=100,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(x_test)))
    print(regressor.best_cost)

predictions = np.mean(predictions_array,axis=0)
print('done')

In [1]:
regressor = rnd_search.best_estimator_

NameError: name 'rnd_search' is not defined

In [29]:
importance = regressor.feature_importances_
print(importance)

[1.15290555e-02 8.94382486e-05 5.85416533e-02 7.72281970e-07
 1.32219863e-06 1.92055393e-04 7.05340176e-02 0.00000000e+00
 1.05808414e-02 2.45398596e-04 0.00000000e+00 7.46702859e-03
 5.96906731e-04 2.67938064e-02 0.00000000e+00 0.00000000e+00
 5.13624302e-06 7.38833958e-07 2.13312581e-03 1.97100380e-02
 1.70334803e-02 5.46415550e-05 8.69472793e-02 3.80817097e-02
 0.00000000e+00 1.54609378e-05 5.12137271e-06 2.81639725e-02
 1.32466612e-04 1.32274990e-06 4.82789032e-02 2.44071233e-05
 3.64244337e-02 0.00000000e+00 0.00000000e+00 1.72834595e-01
 1.63052588e-01 7.80005876e-02 0.00000000e+00 0.00000000e+00
 4.33658950e-06 9.61787509e-06 3.18315797e-03 1.06643793e-02
 0.00000000e+00 9.73447761e-02 4.61812973e-03 6.10718434e-07
 0.00000000e+00 0.00000000e+00 6.71921870e-07 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.68682398e-03 1.51908159e-05]


In [30]:
np.where(importance == 0.) + np.array([1])

array([[ 8, 11, 15, 16, 25, 34, 35, 39, 40, 45, 49, 50, 52, 53, 54]],
      dtype=int64)

In [13]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0) )

The CV score is 1.23355


In [31]:
y_pred = regressor.predict(x_test)

In [32]:
print(r2_score(y_test,y_pred))
print(lg_nrmse(y_test,y_pred))


0.019435484074447427
1.9723936491078695


In [ ]:
0.017777500222701043
1.9778027773294615

In [33]:
test_x = pd.read_csv('./data/test.csv').drop(columns=['ID'])

In [39]:
preds = regressor.predict(test_x.to_numpy())

In [40]:
submit = pd.read_csv('./data/sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')
submit.to_csv('./data/submit.csv', index=False)

Done.
